# 协同过滤

## 基本思想
协同过滤基于用户行为的相似性进行推荐。例如，如果用户A和用户B都喜欢观看mygo、gbc和kon，而用户A还观看了mujica，那么系统很可能会向用户B推荐mujica。

## 核心问题
1. 如何形式化定义用户和物品的交互矩阵？
2. 如何定义用户之间的相似度？
3. 如何计算用户对特定物品的推荐得分？

## 解决方案
### 1. 交互矩阵
通常使用0/1共现矩阵表示用户与物品的交互，1表示交互过，0表示未交互。

### 2. 用户相似度计算方法

#### 余弦相似度 (Cosine Similarity)
余弦相似度计算两个向量夹角的余弦值，范围在[-1,1]之间。对于评分向量通常在[0,1]范围内。余弦值越接近1，表示两个用户的喜好方向越相似。

公式：cos(θ) = (A·B)/(|A|·|B|)

特点：
- 计算简单高效
- 只关注方向相似性，忽略评分尺度差异
- 对零值敏感（未评分项目）
- 不考虑用户评分习惯（有人普遍给高分，有人偏低分）

#### 皮尔逊系数 (Pearson Correlation)
测量两个用户评分模式的线性相关性，范围在[-1,1]之间。1表示完全正相关，-1表示完全负相关，0表示无相关。

公式：r = cov(X,Y)/(σX·σY)

特点：
- 自动补偿评分尺度差异（通过减去平均评分）
- 对用户评分偏好不敏感
- 只考虑两用户共同评分的项目
- 在实际推荐系统中通常优于余弦相似度
- 受离群值影响较大

### 3. 推荐得分计算
基于相似用户的评分加权平均，权重为用户相似度。

# 缺点
- 维护矩阵开销大
- 用户的数据向量稀疏



In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

class UserBasedCF:
    """基于用户的协同过滤推荐算法实现"""
    
    def __init__(self, similarity_method='pearson'):
        """
        初始化推荐系统
        
        参数:
            similarity_method: 相似度计算方法，'pearson'或'cosine'
        """
        self.user_item_matrix = None
        self.similarity_matrix = None
        self.similarity_method = similarity_method
        
    def fit(self, user_item_matrix):
        """
        训练模型
        
        参数:
            user_item_matrix: 用户-物品交互矩阵，行为用户，列为物品
        """
        self.user_item_matrix = user_item_matrix
        
        # 计算用户相似度矩阵
        if self.similarity_method == 'cosine':
            self.similarity_matrix = cosine_similarity(self.user_item_matrix)
        elif self.similarity_method == 'pearson':
            n_users = self.user_item_matrix.shape[0]
            self.similarity_matrix = np.zeros((n_users, n_users))
            
            # 计算每对用户之间的皮尔逊相关系数
            for i in range(n_users):
                for j in range(i, n_users):
                    if i == j:
                        self.similarity_matrix[i, j] = 1.0
                    else:
                        # 获取两个用户都评分过的物品
                        mask = np.logical_and(
                            self.user_item_matrix[i] > 0,
                            self.user_item_matrix[j] > 0
                        )
                        
                        # 确保至少有两个共同评分的物品，否则pearsonr会报错
                        if np.sum(mask) >= 2:
                            # 计算皮尔逊相关系数
                            corr, _ = pearsonr(
                                self.user_item_matrix[i, mask],
                                self.user_item_matrix[j, mask]
                            )
                            self.similarity_matrix[i, j] = corr
                            self.similarity_matrix[j, i] = corr
                        else:
                            # 如果共同评分的物品少于2个，设置相似度为0
                            self.similarity_matrix[i, j] = 0
                            self.similarity_matrix[j, i] = 0
        
        return self
    
    def recommend(self, user_id, top_n=5, n_neighbors=10):
        """
        为指定用户推荐物品
        
        参数:
            user_id: 用户ID
            top_n: 推荐物品数量
            n_neighbors: 考虑的相似用户数量
            
        返回:
            推荐物品的索引列表
        """
        # 获取用户的评分向量
        user_ratings = self.user_item_matrix[user_id]
        
        # 找出用户未评分的物品
        unrated_items = np.where(user_ratings == 0)[0]
        
        # 获取最相似的n个用户（不包括自己）
        similar_users = np.argsort(self.similarity_matrix[user_id])[::-1]
        similar_users = similar_users[similar_users != user_id][:n_neighbors]
        
        # 计算预测评分
        predictions = []
        for item_id in unrated_items:
            # 找出对该物品有评分的相似用户
            mask = self.user_item_matrix[similar_users, item_id] > 0
            sim_users = similar_users[mask]
            
            if len(sim_users) > 0:
                # 计算加权评分
                item_ratings = self.user_item_matrix[sim_users, item_id]
                sim_weights = self.similarity_matrix[user_id, sim_users]
                
                # 避免除以零
                if np.sum(np.abs(sim_weights)) > 0:
                    pred_rating = np.sum(item_ratings * sim_weights) / np.sum(np.abs(sim_weights))
                else:
                    pred_rating = 0
                    
                predictions.append((item_id, pred_rating))
        
        # 按预测评分排序并返回top_n个推荐
        predictions.sort(key=lambda x: x[1], reverse=True)
        return [item_id for item_id, _ in predictions[:top_n]]

# 示例使用
if __name__ == "__main__":
    # 创建一个示例用户-物品矩阵
    data = np.array([
        [5, 3, 0, 1, 0],
        [4, 0, 0, 1, 2],
        [1, 1, 0, 5, 0],
        [0, 0, 4, 0, 4],
        [0, 2, 5, 0, 0]
    ])
    print(data)
    # 初始化并训练模型
    cf = UserBasedCF(similarity_method='pearson')
    cf.fit(data)
    for i in range(data.shape[0]):
        recommendations = cf.recommend(user_id=i, top_n=2)
        print(f"为用户{i}推荐的物品索引: {recommendations}")


[[5 3 0 1 0]
 [4 0 0 1 2]
 [1 1 0 5 0]
 [0 0 4 0 4]
 [0 2 5 0 0]]
为用户0推荐的物品索引: [4, 2]
为用户1推荐的物品索引: [1, 2]
为用户2推荐的物品索引: [2, 4]
为用户3推荐的物品索引: [0, 1]
为用户4推荐的物品索引: [0, 3]
